<a href="https://colab.research.google.com/github/plaban1981/Feature-Engineering-Techniques/blob/master/Comparision_of_categorical_variable_encodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feature-engine

  Created wheel for numpydoc: filename=numpydoc-0.9.2-cp36-none-any.whl size=31893 sha256=5aec2400af1dfff9c498087179d31b52754552bee352a98091cda177cce8964a
  Stored in directory: /root/.cache/pip/wheels/96/f3/52/25c8e1f40637661d27feebc61dae16b84c7cdd93b8bc3d7486
Successfully built numpydoc


In [0]:
import pandas as pd
import numpy as np
import feature_engine


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

Comapre these encoding techniques:

* One hot Encoding

* Replacing labels with Count

* Ordering labels according to target

* Mean Encoding

* Weight Of Evidence

In [4]:
from google.colab import files
files.upload()

Saving titanic.csv to titanic.csv


{'titanic.csv': b'PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked\n1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S\n2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C\n3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S\n4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S\n5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S\n6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q\n7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S\n8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.075,,S\n9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.1333,,S\n10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.0708,,C\n11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4,1,1,PP 9549,16.7,G6,S\n12,1,1,"Bonnell, Miss. Elizabeth",female,58,0,0,113783,26.55,C103,S\n13,0,3,"Saundercock, Mr

In [39]:
data = pd.read_csv('titanic.csv')
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [40]:
cols = ['Pclass','Age', 'SibSp','Parch','Sex','Cabin','Fare', 'Embarked','Survived']
data = data[cols]
data.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked,Survived
0,3,22.0,1,0,male,NaN,7.2500,S,0
1,1,38.0,1,0,female,C85,71.2833,C,1
2,3,26.0,0,0,female,NaN,7.9250,S,1
3,1,35.0,1,0,female,C123,53.1000,S,1
4,3,35.0,0,0,male,NaN,8.0500,S,0


In [41]:
data.isnull().sum()[data.isnull().sum() > 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [0]:
data = data[~data['Embarked'].isnull()]

In [43]:
data.isnull().sum()[data.isnull().sum() > 0]

Age      177
Cabin    687
dtype: int64

In [0]:
data['Cabin'] = data['Cabin'].astype(str).str[0]

In [45]:
data.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked,Survived
0,3,22.0,1,0,male,n,7.2500,S,0
1,1,38.0,1,0,female,C,71.2833,C,1
2,3,26.0,0,0,female,n,7.9250,S,1
3,1,35.0,1,0,female,C,53.1000,S,1
4,3,35.0,0,0,male,n,8.0500,S,0


In [46]:
data['Cabin'].value_counts()

n    687
C     59
B     45
D     33
E     32
A     15
F     13
G      4
T      1
Name: Cabin, dtype: int64

remove data observations with Cabin = T as they are too few

In [0]:
data = data[data['Cabin'] != 'T']

In [0]:
X_train,X_test,y_train,y_test = train_test_split(data.drop('Survived',axis=1),data['Survived'],test_size=0.3,random_state=1)

In [49]:
X_train.shape,X_test.shape

((621, 8), (267, 8))

Replace null values in numerical features by median

In [0]:
def impute_na(df,variable,value):
  df[variable] = df[variable].fillna(value)

impute_na(X_test,'Age',X_train['Age'].median())
impute_na(X_train,'Age',X_train['Age'].median())


In [51]:
X_train.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked
77,3,28.0,0,0,male,n,8.0500,S
876,3,20.0,0,0,male,n,9.8458,S
735,3,28.5,0,0,male,n,16.1000,S
133,2,29.0,1,0,female,n,26.0000,S
476,2,34.0,1,0,male,n,21.0000,S


## 1.1 One Hot Encoding

In [0]:
cat_cols = [cols for cols in X_train.columns if X_train[cols].dtypes == 'O']
num_cols = [cols for cols in X_train.columns if X_train[cols].dtypes != 'O']

In [0]:
def get_OHE(df):
  df_OHE = pd.concat([df[num_cols],pd.get_dummies(df[cat_cols],drop_first=True)],axis=1)
  return df_OHE

In [0]:
X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

In [56]:
X_train_OHE.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_n,Embarked_Q,Embarked_S
77,3,28.0,0,0,8.0500,1,0,0,0,0,0,0,1,0,1
876,3,20.0,0,0,9.8458,1,0,0,0,0,0,0,1,0,1
735,3,28.5,0,0,16.1000,1,0,0,0,0,0,0,1,0,1
133,2,29.0,1,0,26.0000,0,0,0,0,0,0,0,1,0,1
476,2,34.0,1,0,21.0000,1,0,0,0,0,0,0,1,0,1


## 1.2 Count Encoding

In [0]:
def categorical_to_count(train,test):
  df_train = train.copy()
  df_test = test.copy()
  for col in cat_cols:
    counts_map = df_train[col].value_counts().to_dict()
    df_train[col] = df_train[col].map(counts_map)
    df_test[col] = df_test[col].map(counts_map)
  return df_train,df_test

In [0]:
X_train_count,X_test_count = categorical_to_count(X_train,X_test)

In [60]:
X_train['Embarked'].value_counts().to_dict()

{'C': 117, 'Q': 50, 'S': 454}

In [61]:
X_train['Sex'].value_counts().to_dict()

{'female': 213, 'male': 408}

In [62]:
X_train['Cabin'].value_counts().to_dict()

{'A': 11, 'B': 31, 'C': 42, 'D': 16, 'E': 26, 'F': 11, 'G': 3, 'n': 481}

In [59]:
X_train_count.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked
77,3,28.0,0,0,408,481,8.0500,454
876,3,20.0,0,0,408,481,9.8458,454
735,3,28.5,0,0,408,481,16.1000,454
133,2,29.0,1,0,213,481,26.0000,454
476,2,34.0,1,0,408,481,21.0000,454


## 1.3 Ordered Integer Encoding

In [0]:
def categories_to_ordered(train,test,y_train,y_test):
  df_train = pd.concat([train,y_train],axis=1).copy()
  df_test = pd.concat([test,y_test],axis=1).copy()
  for col in cat_cols:
    ordered_labels = df_train.groupby([col])['Survived'].mean().sort_values().index

    ordinal_label = {k:i for i,k in enumerate(ordered_labels,0)}

    df_train[col] = df_train[col].map(ordinal_label)
    df_test[col] = df_test[col].map(ordinal_label)

  #remove target
  df_train.drop('Survived',axis=1,inplace=True)
  df_test.drop('Survived',axis=1,inplace=True)

  return df_train,df_test


In [0]:
X_train_ordered,X_test_ordered = categories_to_ordered(X_train,X_test,y_train,y_test)

In [65]:
X_train_ordered.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked
77,3,28.0,0,0,0,0,8.0500,0
876,3,20.0,0,0,0,0,9.8458,0
735,3,28.5,0,0,0,0,16.1000,0
133,2,29.0,1,0,1,0,26.0000,0
476,2,34.0,1,0,0,0,21.0000,0


##1.4 Mean Encoding

In [0]:
def categories_to_mean(train,test,y_train,y_test):
  df_train = pd.concat([train,y_train],axis=1).copy()
  df_test = pd.concat([test,y_test],axis=1).copy()
  for col in cat_cols:
    ordered_labels = df_train.groupby([col])['Survived'].mean().to_dict()

    df_train[col] = df_train[col].map(ordered_labels)
    df_test[col] = df_test[col].map(ordered_labels)

  #remove target
  df_train.drop('Survived',axis=1,inplace=True)
  df_test.drop('Survived',axis=1,inplace=True)

  return df_train,df_test

In [0]:
X_train_mean,X_test_mean = categories_to_mean(X_train,X_test,y_train,y_test)

In [69]:
X_train_mean.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked
77,3,28.0,0,0,0.203431,0.31185,8.0500,0.337004
876,3,20.0,0,0,0.203431,0.31185,9.8458,0.337004
735,3,28.5,0,0,0.203431,0.31185,16.1000,0.337004
133,2,29.0,1,0,0.746479,0.31185,26.0000,0.337004
476,2,34.0,1,0,0.203431,0.31185,21.0000,0.337004


## 1.5 WoE : Weight of Evidence

In [0]:
def categories_to_woe(train,test,y_train,y_test):
  df_train = pd.concat([train,y_train],axis=1).copy()
  df_test = pd.concat([test,y_test],axis=1).copy()

  for col in cat_cols:
    prob_df = pd.DataFrame(df_train.groupby([col])['Survived'].mean())

    prob_df['died'] = 1 - prob_df['Survived']

    prob_df['WoE']  = np.log(prob_df['Survived']/prob_df['died'])
    woe = prob_df['WoE'].to_dict()

    df_train[col] = df_train[col].map(woe)
    df_test[col] = df_test[col].map(woe)

  df_train.drop('Survived',axis=1,inplace=True)
  df_test.drop('Survived',axis=1,inplace=True)

  return df_train,df_test


In [0]:
X_train_woe,X_test_woe = categories_to_woe(X_train,X_test,y_train,y_test)

In [75]:
X_train_woe.head()

,Pclass,Age,SibSp,Parch,Sex,Cabin,Fare,Embarked
77,3,28.0,0,0,-1.364985,-0.791483,8.0500,-0.676672
876,3,20.0,0,0,-1.364985,-0.791483,9.8458,-0.676672
735,3,28.5,0,0,-1.364985,-0.791483,16.1000,-0.676672
133,2,29.0,1,0,1.079920,-0.791483,26.0000,-0.676672
476,2,34.0,1,0,-1.364985,-0.791483,21.0000,-0.676672


## Random_Forest Performance

In [0]:
def run_random_forest(X_train,X_test,y_train,y_test):
  rf = RandomForestClassifier(n_estimators=100,random_state=1,max_depth=3)
  rf.fit(X_train,y_train)
  print('Train Set')
  pred = rf.predict_proba(X_train)
  print('Random Forest roc-auc {}'.format(roc_auc_score(y_train,pred[:,1])))
  print('Test Set')
  pred = rf.predict_proba(X_test)
  print('Random Forest roc-auc {}'.format(roc_auc_score(y_test,pred[:,1])))

In [79]:
run_random_forest(X_train_OHE,X_test_OHE,y_train,y_test)

Train Set
Random Forest roc-auc 0.8654080987374344
Test Set
Random Forest roc-auc 0.8676488346818017


In [80]:
run_random_forest(X_train_count,X_test_count,y_train,y_test)

Train Set
Random Forest roc-auc 0.8744139645434921
Test Set
Random Forest roc-auc 0.8820794590025359


In [81]:
run_random_forest(X_train_ordered,X_test_ordered,y_train,y_test)

Train Set
Random Forest roc-auc 0.8733073115419002
Test Set
Random Forest roc-auc 0.8784567081270378


In [82]:
run_random_forest(X_train_mean,X_test_mean,y_train,y_test)

Train Set
Random Forest roc-auc 0.8733073115419002
Test Set
Random Forest roc-auc 0.8784567081270378


In [83]:
run_random_forest(X_train_woe,X_test_woe,y_train,y_test)

Train Set
Random Forest roc-auc 0.8733073115419002
Test Set
Random Forest roc-auc 0.8784567081270378


## The worst roc-auc-score was given by OHE as  for tree based classifiers high feature space is decremental .

## Logistic Regression

In [0]:
def run_logistic_regression(X_train,X_test,y_train,y_test):
  lr = LogisticRegression(C=0.01,random_state=1,solver='liblinear')
  lr.fit(X_train,y_train)
  print('Train Set')
  pred = lr.predict_proba(X_train)
  print('Random Forest roc-auc {}'.format(roc_auc_score(y_train,pred[:,1])))
  print('Test Set')
  pred = lr.predict_proba(X_test)
  print('Random Forest roc-auc {}'.format(roc_auc_score(y_test,pred[:,1])))

In [90]:
run_logistic_regression(X_train_OHE,X_test_OHE,y_train,y_test)

Train Set
Random Forest roc-auc 0.822940971237925
Test Set
Random Forest roc-auc 0.8083564786861489


In [91]:
run_logistic_regression(X_train_count,X_test_count,y_train,y_test)

Train Set
Random Forest roc-auc 0.7819675527159335
Test Set
Random Forest roc-auc 0.8232097572756913


In [92]:
run_logistic_regression(X_train_ordered,X_test_ordered,y_train,y_test)

Train Set
Random Forest roc-auc 0.8175767025011447
Test Set
Random Forest roc-auc 0.7996618765849535


In [93]:
run_logistic_regression(X_train_mean,X_test_mean,y_train,y_test)

Train Set
Random Forest roc-auc 0.7817167840554744
Test Set
Random Forest roc-auc 0.7560681077164593


In [94]:
run_logistic_regression(X_train_woe,X_test_woe,y_train,y_test)

Train Set
Random Forest roc-auc 0.8414433371857214
Test Set
Random Forest roc-auc 0.8541842772612003
